In [16]:
import ssl
import utm
import pandas as pd
import numpy as np
import geocoder
import folium
import json
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Proj, transform

**Cargamos primero el picke**

In [2]:
df = pd.read_pickle("accidentes_pkl.pkl")

**La idea es "limpiar" la columna localización de caracteres "raros" y separar calles que son cruces**

In [3]:
df['localizacion']

0         calle de cartagena num                        ...
1         calle de cartagena num                        ...
2         calle de cartagena num                        ...
3         autovia  m-30 calzada 2 km.                   ...
4         autovia  m-30 calzada 2 km.                   ...
                                ...                        
478778         call. virgen de lluc / call. buen gobernador
478779         call. virgen de lluc / call. buen gobernador
478780                                paseo. juan xxiii, 16
478781                                paseo. juan xxiii, 16
478782                         avda. monforte de lemos, 172
Name: localizacion, Length: 478783, dtype: object

In [4]:
# Eliminamos utilizando regex diferentes simbolos y datos de la columna localización 
df['localizacion'] = df['localizacion'].str.replace(r'[,]?\d+$', '', regex=True)
df.loc[:, 'localizacion'] = df.loc[:, 'localizacion'].str.replace('call.', 'calle', regex=False)
df.loc[:, 'localizacion'] = df.loc[:, 'localizacion'].str.replace('num', '', regex=False)
df.loc[:, 'localizacion'] = df.loc[:, 'localizacion'].str.replace(',', '', regex=False)
df.loc[:, 'localizacion'] = df.loc[:, 'localizacion'].str.replace('c/', '', regex=False)
df.loc[:, 'localizacion'] = df.loc[:, 'localizacion'].str.replace('c/ ', '', regex=False)
df.loc[:, 'localizacion'] = df.loc[:, 'localizacion'].str.replace('/', '-', regex=False)
df.loc[:, 'numero'] = df.loc[:, 'numero'].str.replace('0', '', regex=False)
df

,accidente_id,fecha,rango_horario,localizacion,numero,distrito,coordenada_x,coordenada_y,condicion,lesividad,persona_implicada,positiva_alcohol,positiva_droga,rango_edad,sexo,tipo_accidente,tipo_vehiculo,victimas
0,2010.135000,2010-01-01,0,calle de cartagena ...,14,chamartin,NaN,NaN,desfavorable,ingreso inferior 24h,conductor,NaN,NaN,5.0,hombre,colision doble,turismo,1.0
1,2010.135000,2010-01-01,0,calle de cartagena ...,14,chamartin,NaN,NaN,desfavorable,ingreso inferior 24h,conductor,NaN,NaN,12.0,hombre,colision doble,turismo,1.0
2,2010.135000,2010-01-01,0,calle de cartagena ...,14,chamartin,NaN,NaN,desfavorable,asistencia sanitaria sin ingreso,pasajero,NaN,NaN,10.0,mujer,colision doble,turismo,1.0
3,2010.940000,2010-01-01,1,autovia m-30 calzada 2 km. ...,12,puente de vallecas,NaN,NaN,favorable,asistencia sanitaria sin ingreso,pasajero,NaN,NaN,7.0,mujer,colision multiple,NaN,7.0
4,2010.940000,2010-01-01,1,autovia m-30 calzada 2 km. ...,12,puente de vallecas,NaN,NaN,favorable,asistencia sanitaria sin ingreso,pasajero,NaN,NaN,8.0,mujer,colision multiple,NaN,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478778,2024.004705,2024-01-31,9,calle virgen de lluc - calle buen gobernador,12,ciudad lineal,444464.372,4476185.781,favorable,asistencia sanitaria sin ingreso,conductor,0.0,NaN,8.0,hombre,colision doble,bicicleta,NaN
478779,2024.004705,2024-01-31,9,calle virgen de lluc - calle buen gobernador,12,ciudad lineal,444464.372,4476185.781,favorable,sin asistencia sanitaria,conductor,0.0,NaN,9.0,mujer,colision doble,turismo,NaN
478780,2024.005341,2024-01-27,13,paseo. juan xxiii,16,chamberi,439084.688,4477567.644,favorable,NaN,conductor,0.0,NaN,7.0,mujer,colision doble,turismo,NaN
478781,2024.005341,2024-01-27,13,paseo. juan xxiii,16,chamberi,439084.688,4477567.644,favorable,NaN,conductor,0.0,NaN,16.0,hombre,colision doble,turismo,NaN


**Ahora vamos a crear una nueva columna que se llamará localización completa sumando localizacion y numero**

El objetivo de hacer esto es para ir identificando localizaciones que se repitan con el objetivo de buscar puntos negros

In [5]:
def combinar_localizacion_numero(row):
    # Combinamos 'localizacion' y 'numero' si 'numero' no es NaN ni está vacío
    if pd.notna(row['numero']) and row['numero'] != '':
        return f"{row['localizacion']}, {row['numero']}"
    # Devolver solo 'localizacion' si 'numero' es NaN o está vacío
    else:
        return row['localizacion']

df['localizacion_completa'] = df.apply(combinar_localizacion_numero, axis=1)

**Ahora lo que vamos a hacer es separar las direcciones que son cruces para buscar coordenadas**

In [6]:
df = df.copy()
# Vamos a dividir la columna 'localizacion' en dos nuevas columnas 'direccion_1' y 'direccion_2'
# usando el guión como separador la función expand=True hará que el resultado se divida en columnas separadas
direcciones = df['localizacion'].str.split(' - ', expand=True)

df['direccion_1'] = direcciones[0]
df['direccion_2'] = direcciones[1] if direcciones.shape[1] > 1 else None


**Aquí lo que hacemos es eliminar los duplicados de accidente_id, ya que lo que queremos buscar son los puntos_negros**

In [7]:
df_unicas = df.drop_duplicates(subset='accidente_id')
print(df_unicas)

        accidente_id      fecha  rango_horario  \
0        2010.135000 2010-01-01              0   
3        2010.940000 2010-01-01              1   
11       2010.700000 2010-01-01              2   
13       2010.400000 2010-01-01              4   
24       2010.172000 2010-01-01              5   
...              ...        ...            ...   
478774   2024.004340 2024-01-18             12   
478776   2024.004634 2024-01-30             18   
478778   2024.004705 2024-01-31              9   
478780   2024.005341 2024-01-27             13   
478782   2024.006301 2024-01-29             21   

                                             localizacion numero  \
0       calle de cartagena                            ...     14   
3       autovia  m-30 calzada 2 km.                   ...     12   
11      glorieta de emilio castelar - paseo de la cast...          
13      autovia  m-30 calzada 1 km.                   ...     85   
24      calle de o'donnell - calle del doctor esquerdo...  

**A continuación lo que vamos a hacer es ver es en que puntos se repiten los accidentes en Madrid**

Para ello vamos a contar las veces que se repite localizacion_completa, que es el lugar exacto del accidente.
Y vamos a crear una nueva columna que se llama puntos_negros para que cuente las veces que existen accidentes en ese punto

In [8]:
# Paso 1: Contamos los valores repetidos en 'localizacion_completa'
conteo_direcciones = df_unicas['localizacion_completa'].value_counts()

# Paso 2: Unir la suma de direcciones de df_unicas basado en 'localizacion_completa'
df_unicas = df_unicas.merge(conteo_direcciones.rename('punto_negro'), left_on='localizacion_completa', right_index=True)

# Paso 3: Crear df_puntos_negros con todas las columnas de df_unicas y ordenarlo por 'punto_negro'
df_puntos_negros = df_unicas.copy()
df_puntos_negros = df_puntos_negros.sort_values(by='punto_negro', ascending=False)

df_puntos_negros



,accidente_id,fecha,rango_horario,localizacion,numero,distrito,coordenada_x,coordenada_y,condicion,lesividad,...,positiva_droga,rango_edad,sexo,tipo_accidente,tipo_vehiculo,victimas,localizacion_completa,direccion_1,direccion_2,punto_negro
105052,2013.107380,2013-12-01,20,paseo de la castellana - plaza de lima ...,,tetuan,NaN,NaN,favorable,ingreso superior 24h,...,NaN,8.0,hombre,atropello,NaN,1.0,paseo de la castellana - plaza de lima ...,paseo de la castellana,plaza de lima ...,132
54211,2012.217000,2012-01-05,18,paseo de la castellana - plaza de lima ...,,tetuan,NaN,NaN,favorable,asistencia sanitaria sin ingreso,...,NaN,15.0,hombre,atropello,NaN,2.0,paseo de la castellana - plaza de lima ...,paseo de la castellana,plaza de lima ...,132
203286,2017.447800,2017-05-11,23,paseo de la castellana - plaza de lima ...,,chamartin,NaN,NaN,favorable,asistencia sanitaria sin ingreso,...,NaN,9.0,hombre,colision doble,ciclomotor,1.0,paseo de la castellana - plaza de lima ...,paseo de la castellana,plaza de lima ...,132
213460,2017.877300,2017-09-20,21,paseo de la castellana - plaza de lima ...,,tetuan,NaN,NaN,favorable,asistencia sanitaria sin ingreso,...,NaN,8.0,mujer,colision doble,turismo,2.0,paseo de la castellana - plaza de lima ...,paseo de la castellana,plaza de lima ...,132
252919,2018.166570,2018-12-30,0,paseo de la castellana - plaza de lima ...,,chamartin,NaN,NaN,favorable,ingreso inferior 24h,...,NaN,9.0,hombre,colision doble,turismo,1.0,paseo de la castellana - plaza de lima ...,paseo de la castellana,plaza de lima ...,132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359217,2021.013625,2021-07-30,12,autov. m-30 +00000e m-30 calzada 2ª pk 0.,+e,ciudad lineal,442883.453,4481279.846,NaN,sin asistencia sanitaria,...,NaN,7.0,hombre,colision doble,camion,NaN,"autov. m-30 +00000e m-30 calzada 2ª pk 0., +e",autov. m-30 +00000e m-30 calzada 2ª pk 0.,None,1
225439,2018.116400,2018-01-30,14,calle de villablanca ...,1,vicalvaro,NaN,NaN,favorable,asistencia sanitaria sin ingreso,...,NaN,7.0,hombre,choque con objeto fijo,turismo,1.0,calle de villablanca ...,calle de villablanca ...,None,1
225435,2018.115200,2018-01-30,13,calle de marcelo usera ...,87,usera,NaN,NaN,favorable,ingreso superior 24h,...,NaN,2.0,hombre,atropello,NaN,1.0,calle de marcelo usera ...,calle de marcelo usera ...,None,1
359228,2021.013628,2021-07-30,13,calle almaden - calle san pedro,6,centro,441042.895,4473621.725,favorable,sin asistencia sanitaria,...,NaN,12.0,hombre,colision doble,turismo,NaN,"calle almaden - calle san pedro, 6",calle almaden,calle san pedro,1


**Aquí lo que hacemos es utilizando la columna de puntos negros ordenarlos por años, así vemos cada año donde es
el lugar de top de punto negro y ordenado por barrio**

Como hay muchos datos que no nos van a interesar para nada le digo que me coja como mucho 1.000 datos por barrio, 
siendo el mínimo 30. Habrá barrios donde hay menos puntos negros...

In [9]:
resultados = []

for año in range(2010, 2024):
    df_año = df_puntos_negros[df_puntos_negros['fecha'].dt.year == año]
    
    # Esto es para ordenar por 'punto_negro' de manera descendente
    df_año_ordenado = df_año.sort_values(by='punto_negro', ascending=False)
        
    registros_seleccionados = []
    
    # Agrupamos
    for distrito, grupo in df_año_ordenado.groupby('distrito'):
        registros_distrito = grupo.head(30)  # Aquí le decimos que coja los 30 primeros de cada distrito (si los hay)
        registros_seleccionados.append(registros_distrito)
    
    # Combinamos
    df_seleccionados = pd.concat(registros_seleccionados).drop_duplicates().head(1000)  # Limitamos a 1.000 registros, no queremos más para nada
    resultados.append(df_seleccionados)

# Combinamos todo
df_resultado_final = pd.concat(resultados)

# Y para el año 2024 como habían muy pocos, pues los añadimos todos
df_2024 = df_puntos_negros[df_puntos_negros['fecha'].dt.year == 2024]
df_resultado_final = pd.concat([df_resultado_final, df_2024])

df_resultado_final

,accidente_id,fecha,rango_horario,localizacion,numero,distrito,coordenada_x,coordenada_y,condicion,lesividad,...,positiva_droga,rango_edad,sexo,tipo_accidente,tipo_vehiculo,victimas,localizacion_completa,direccion_1,direccion_2,punto_negro
22083,2010.995700,2010-11-09,4,autovia m-30 calzada 1 km. ...,12,arganzuela,NaN,NaN,desfavorable,asistencia sanitaria sin ingreso,...,NaN,9.0,hombre,choque con objeto fijo,turismo,1.0,autovia m-30 calzada 1 km. ...,autovia m-30 calzada 1 km. ...,None,64
6062,2010.296200,2010-03-31,16,autovia m-30 calzada 1 km. ...,12,arganzuela,NaN,NaN,favorable,asistencia sanitaria sin ingreso,...,NaN,7.0,hombre,colision doble,turismo,1.0,autovia m-30 calzada 1 km. ...,autovia m-30 calzada 1 km. ...,None,64
174,2010.153000,2010-01-04,16,autovia m-30 calzada 1 km. ...,12,arganzuela,NaN,NaN,desfavorable,asistencia sanitaria sin ingreso,...,NaN,6.0,mujer,choque con objeto fijo,turismo,1.0,autovia m-30 calzada 1 km. ...,autovia m-30 calzada 1 km. ...,None,64
20271,2010.922200,2010-10-19,9,autovia m-30 calzada 1 km. ...,12,arganzuela,NaN,NaN,favorable,ingreso inferior 24h,...,NaN,11.0,hombre,colision multiple,turismo,3.0,autovia m-30 calzada 1 km. ...,autovia m-30 calzada 1 km. ...,None,64
19168,2010.874500,2010-10-03,23,autovia m-30 calzada 1 km. ...,12,arganzuela,NaN,NaN,desfavorable,asistencia sanitaria sin ingreso,...,NaN,6.0,mujer,choque con objeto fijo,turismo,1.0,autovia m-30 calzada 1 km. ...,autovia m-30 calzada 1 km. ...,None,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476601,2024.001546,2024-01-15,17,plaza. gabriel miro,7,centro,439330.129,4473727.919,NaN,NaN,...,NaN,8.0,mujer,colision doble,turismo,NaN,"plaza. gabriel miro , 7",plaza. gabriel miro,None,1
476603,2024.001547,2024-01-15,20,calle nuñez de balboa,86,salamanca,442146.054,4476054.421,favorable,NaN,...,NaN,11.0,mujer,colision doble,turismo,NaN,"calle nuñez de balboa , 86",calle nuñez de balboa,None,1
476605,2024.001548,2024-01-15,19,calle vigil - calle boyer,2a,vicalvaro,450240.390,4472879.964,desfavorable,asistencia sanitaria sin ingreso,...,NaN,10.0,hombre,colision doble,motocicleta,NaN,"calle vigil - calle boyer, 2a",calle vigil,calle boyer,1
476607,2024.001550,2024-01-15,20,autov. m-30 09nc,9nc7,retiro,443468.671,4472932.015,desfavorable,asistencia sanitaria sin ingreso,...,NaN,13.0,hombre,colision doble,motocicleta,NaN,"autov. m-30 09nc, 9nc7",autov. m-30 09nc,None,1


In [10]:
# Vamos a mirar por ejemplo en el distrito centro los resultados para ver si ha funcionado bien. 
df_centro = df_resultado_final.loc[df_resultado_final['distrito'] == 'centro']

# Mostrar los resultados para el distrito 'Centro'
df_centro

# Por ejemplo aquí podemos ver ha cogido 520 datos.


,accidente_id,fecha,rango_horario,localizacion,numero,distrito,coordenada_x,coordenada_y,condicion,lesividad,...,positiva_droga,rango_edad,sexo,tipo_accidente,tipo_vehiculo,victimas,localizacion_completa,direccion_1,direccion_2,punto_negro
10695,2010.496300,2010-06-01,11,calle de alcala - plaza de cibeles ...,,centro,NaN,NaN,favorable,ingreso inferior 24h,...,NaN,10.0,hombre,colision doble,NaN,1.0,calle de alcala - plaza de cibeles ...,calle de alcala,plaza de cibeles ...,84
23468,2010.106250,2010-11-24,14,paseo del prado - plaza de canovas del castill...,,centro,NaN,NaN,favorable,ingreso inferior 24h,...,NaN,11.0,hombre,colision multiple,turismo,1.0,paseo del prado - plaza de canovas del castill...,paseo del prado,plaza de canovas del castillo ...,76
17472,2010.792600,2010-09-09,15,plaza del emperador carlos v ...,1,centro,NaN,NaN,favorable,ingreso inferior 24h,...,NaN,9.0,mujer,colision doble,turismo,1.0,plaza del emperador carlos v ...,plaza del emperador carlos v ...,None,71
12989,2010.592200,2010-06-29,20,plaza del emperador carlos v ...,1,centro,NaN,NaN,favorable,ingreso inferior 24h,...,NaN,8.0,hombre,colision doble,turismo,1.0,plaza del emperador carlos v ...,plaza del emperador carlos v ...,None,71
10321,2010.480100,2010-05-27,16,plaza del emperador carlos v ...,1,centro,NaN,NaN,favorable,ingreso inferior 24h,...,NaN,8.0,hombre,colision doble,turismo,1.0,plaza del emperador carlos v ...,plaza del emperador carlos v ...,None,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476618,2024.001558,2024-01-15,20,calle meson de paredes - calle juanelo,7,centro,440178.731,4473657.333,desfavorable,NaN,...,NaN,6.0,hombre,colision doble,bicicleta,NaN,"calle meson de paredes - calle juanelo, 7",calle meson de paredes,calle juanelo,1
476573,2024.001532,2024-01-15,17,calle angel,19,centro,439511.276,4473493.190,desfavorable,NaN,...,NaN,7.0,hombre,colision doble,turismo,NaN,"calle angel , 19",calle angel,None,1
476587,2024.001538,2024-01-15,18,calle don pedro - plaza. gabriel miro,2,centro,439353.596,4473734.854,NaN,NaN,...,NaN,8.0,hombre,choque con objeto fijo,turismo,NaN,"calle don pedro - plaza. gabriel miro, 2",calle don pedro,plaza. gabriel miro,1
476593,2024.001542,2024-01-15,20,calle ribera de curtidores,24,centro,439985.807,4473205.539,desfavorable,NaN,...,NaN,11.0,hombre,colision doble,turismo,NaN,"calle ribera de curtidores , 24",calle ribera de curtidores,None,1


**Ahora lo que vamos a hacer es agrupar por localización_completa y ordenamos por punto_negro**

Pero vamos a crear una nueva columna que nos haga de ranking del primero como punto negro hasta el menor

In [11]:
# Primero: Agrupamos por 'localizacion_completa'
agregaciones = {
    'punto_negro': 'max',
    'accidente_id': 'first',
    'fecha': 'first',
    'rango_horario': 'first',
    'localizacion': 'first',
    'numero': 'first',
    'distrito': 'first',
    'coordenada_x': 'first',
    'coordenada_y': 'first',
    'condicion': 'first',
    'lesividad': 'first',
    'persona_implicada': 'first',
    'positiva_alcohol': 'first',
    'positiva_droga': 'first',
    'rango_edad': 'first',
    'sexo': 'first',
    'tipo_accidente': 'first',
    'tipo_vehiculo': 'first',
    'victimas': 'first',
    'direccion_1': 'first',
    'direccion_2': 'first'
}

df_agrupado = df_resultado_final.groupby('localizacion_completa', as_index=False).agg(agregaciones)

# Segundo: Ordenamos por 'punto_negro' de manera descendente
df_agrupado = df_agrupado.sort_values(by='punto_negro', ascending=False)

# Tercero: Creams la columna 'ranking_punto_negro'
df_agrupado['ranking_punto_negro'] = range(1, len(df_agrupado) + 1)

df_agrupado

#Así tenemos ordenados del punto más peligroso de Madrid hacia menos... aunque siguen siendo punto negro

,localizacion_completa,punto_negro,accidente_id,fecha,rango_horario,localizacion,numero,distrito,coordenada_x,coordenada_y,...,positiva_alcohol,positiva_droga,rango_edad,sexo,tipo_accidente,tipo_vehiculo,victimas,direccion_1,direccion_2,ranking_punto_negro
1865,paseo de la castellana - plaza de lima ...,132,2010.506300,2010-06-05,21,paseo de la castellana - plaza de lima ...,,chamartin,NaN,NaN,...,NaN,NaN,5.0,mujer,colision doble,turismo,2.0,paseo de la castellana,plaza de lima ...,1
64,"autov. m-30 +01100i, +11i",121,2019.017297,2019-03-27,12,autov. m-30 +01100i,+11i,arganzuela,442724.319,4471669.860,...,0.0,NaN,11.0,hombre,colision multiple,furgoneta-autocaravana,NaN,autov. m-30 +01100i,None,2
11,"autov. a-2 +00500e, +5e",118,2019.037143,2019-11-29,22,autov. a-2 +00500e,+5e,ciudad lineal,444536.890,4477584.100,...,0.0,NaN,7.0,mujer,colision multiple,turismo,NaN,autov. a-2 +00500e,None,3
1734,complejo aeropuerto de barajas ...,116,2010.331000,2010-01-12,8,complejo aeropuerto de barajas ...,,barajas,NaN,NaN,...,NaN,NaN,8.0,mujer,choque con objeto fijo,turismo,1.0,complejo aeropuerto de barajas ...,None,4
1864,paseo de la castellana - plaza de cuzco ...,112,2010.138200,2010-02-12,13,paseo de la castellana - plaza de cuzco ...,,chamartin,NaN,NaN,...,NaN,NaN,7.0,mujer,colision multiple,turismo,2.0,paseo de la castellana,plaza de cuzco ...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,"calle eugenio d'ors , 2",1,2024.000251,2024-01-04,17,calle eugenio d'ors,2,moratalaz,444309.357,4473661.560,...,0.0,NaN,NaN,None,colision doble,turismo,NaN,calle eugenio d'ors,None,2081
1088,"calle estrella polar - calle pez austral, 24",1,2024.000800,2024-01-10,7,calle estrella polar - calle pez austral,24,retiro,443533.577,4473608.565,...,0.0,NaN,8.0,hombre,colision doble,motocicleta,NaN,calle estrella polar,calle pez austral,2082
1087,"calle estrella hadar - calle eros, 6",1,2024.003595,2024-01-31,15,calle estrella hadar - calle eros,6,arganzuela,442272.212,4471688.482,...,0.0,NaN,10.0,mujer,colision doble,turismo,NaN,calle estrella hadar,calle eros,2083
1086,"calle estefanita , 3",1,2024.001504,2024-01-14,20,calle estefanita,3,villaverde,442600.472,4466540.705,...,0.0,NaN,9.0,mujer,atropello,turismo,NaN,calle estefanita,None,2084


**Aquí contamos los valores NaN para saber las coordenadas que tenemos que buscar**

In [12]:
nan_coordenada_x = df_resultado_final['coordenada_x'].isna().sum()
nan_coordenada_y = df_resultado_final['coordenada_y'].isna().sum()

print(f"Número de valores NaN en 'coordenada_x': {nan_coordenada_x}")
print(f"Número de valores NaN en 'coordenada_y': {nan_coordenada_y}")

Número de valores NaN en 'coordenada_x': 5672
Número de valores NaN en 'coordenada_y': 5672


# Ojo que tarda al ejecutar
## No ejecutar si no se quiere estar un buen rato esperando...

**Creamos diferentes funciones para a través de geocoder (pip install geocoder) buscar las coordenadas en UTM que son
las que existe en nuestro dataframe.**

Hacemos un bucle y que itere en cada fila que no contenga ya coordenada (que son principalmente del año 2010 al 2018).
Es importante destacar que es probable que no encuentre alguna coordenada.
Lo que hacemos en el for es buscar primero en localizacion_completa. En caso de no encontrar que busque direccion_1 y direccion_2 la coordenada y haga como un cruce de coordenadas, y en caso de no encontrar tampoco que busque en 
direccion_1 + columna numero. 

Como se repetía el proceso para direcciones que eran iguales, lo que hacemos es reducir a direcciones únicas y luego que las copie al resto que sean iguales las coordenadas. Para ahorrar tiempo.

In [13]:
def obtener_coordenadas_calle(direccion):
    g = geocoder.osm(direccion)
    if g.ok:
        return g.lat, g.lng
    return None, None

def buscar_coordenadas_utm_geocoder(direccion):
    lat, lng = obtener_coordenadas_calle(direccion)
    if lat and lng:
        utm_coords = utm.from_latlon(lat, lng)
        return utm_coords[0], utm_coords[1]
    return None, None

def estimar_punto_cruce(calle1, calle2, distrito):
    direccion1 = f"{calle1}, {distrito}, Madrid, Spain"
    direccion2 = f"{calle2}, {distrito}, Madrid, Spain"
    
    lat1, lng1 = obtener_coordenadas_calle(direccion1)
    lat2, lng2 = obtener_coordenadas_calle(direccion2)
    
    if lat1 and lat2 and lng1 and lng2:
        lat_cruce = (lat1 + lat2) / 2
        lng_cruce = (lng1 + lng2) / 2
        utm_cruce = utm.from_latlon(lat_cruce, lng_cruce)
        return utm_cruce[0], utm_cruce[1]
    return None, None

# Creamos un nuevo df con las columnas que nos interesan
df_unicas = df_resultado_final[['localizacion_completa', 'distrito', 'direccion_1', 'direccion_2', 'numero', 'coordenada_x', 'coordenada_y']].drop_duplicates(subset='localizacion_completa')

# Iteramos sobre df_unicas para buscar y actualizar coordenadas donde sea necesario (donde haya Nan)
for index, row in df_unicas.iterrows():
    if pd.isna(row['coordenada_x']) or pd.isna(row['coordenada_y']):
        # Primero buscará 'localizacion_completa'
        coordenada_x, coordenada_y = buscar_coordenadas_utm_geocoder(f"{row['localizacion_completa']}, {row['distrito']}, Madrid, Spain")
        if coordenada_x is not None and coordenada_y is not None:
            print(f"Coordenada encontrada para {row['localizacion_completa']}")
            df_resultado_final.at[index, 'coordenada_x'] = coordenada_x
            df_resultado_final.at[index, 'coordenada_y'] = coordenada_y
        else:
            print(f"Coordenada no encontrada para {row['localizacion_completa']}, intentando con cruce...")

            # Si no lo encuentra, irá a buscar entre el cruce entre 'direccion_1' y 'direccion_2'
            if pd.notna(row['direccion_2']):
                coordenada_x, coordenada_y = estimar_punto_cruce(row['direccion_1'], row['direccion_2'], row['distrito'])
                if coordenada_x is not None and coordenada_y is not None:
                    print(f"Coordenada encontrada para cruce entre {row['direccion_1']} y {row['direccion_2']}")
                    df_resultado_final.at[index, 'coordenada_x'] = coordenada_x
                    df_resultado_final.at[index, 'coordenada_y'] = coordenada_y
                else:
                    print(f"Coordenada no encontrada para cruce, intentando con 'direccion_1' y 'numero'...")

            # Y si no lo encuentra, probamos una última opción de búsqueda que es buscar con 'direccion_1' + 'numero'
            if coordenada_x is None or coordenada_y is None:
                direccion_final = f"{row['direccion_1']}"
                if pd.notna(row['numero']):
                    direccion_final += f" {row['numero']}"
                direccion_final += f", {row['distrito']}, Madrid, Spain"
                coordenada_x, coordenada_y = buscar_coordenadas_utm_geocoder(direccion_final)
                if coordenada_x is not None and coordenada_y is not None:
                    print(f"Coordenada encontrada para {direccion_final}")
                    df_resultado_final.at[index, 'coordenada_x'] = coordenada_x
                    df_resultado_final.at[index, 'coordenada_y'] = coordenada_y
                else:
                    print(f"Coordenada finalmente no encontrada para {direccion_final}")


# Guardamos df_unicas con las coordenadas en un archivo pickle para no tener que ejecutar este paso que tarda mucho
df_unicas.to_pickle('mapas/df_coordenadas.pkl')


Coordenada no encontrada para autovia  m-30 calzada 1 km.                                                                         , 12, intentando con cruce...
Coordenada finalmente no encontrada para autovia  m-30 calzada 1 km.                                                                          12, arganzuela, Madrid, Spain
Coordenada no encontrada para autovia  m-30 calzada 1 km.                                                                         , 17, intentando con cruce...
Coordenada finalmente no encontrada para autovia  m-30 calzada 1 km.                                                                          17, arganzuela, Madrid, Spain
Coordenada no encontrada para autovia  m-30 calzada 1 km.                                                                         , 11, intentando con cruce...
Coordenada finalmente no encontrada para autovia  m-30 calzada 1 km.                                                                          11, arganzuela, Madrid, Spain
Coor

In [19]:
print(df_unicas[df_unicas.duplicated('localizacion_completa', keep=False)])


Empty DataFrame
Columns: [localizacion_completa, distrito, direccion_1, direccion_2, numero, coordenada_x, coordenada_y]
Index: []


In [20]:
print(len(df_resultado_final))


10462


**Aquí lo que vamos a hacer es llevar los datos de df_unicas al df_resultado_final todas las coordenadas que ha encontrado**

In [21]:
#Cargamos el pickle si no has ejecutado
#df_unicas = pd.read_pickle('mapas/df_coordenadas.pkl')

# Crea un índice temporal en df_resultado_final basado en 'localizacion_completa'
df_resultado_final.set_index('localizacion_completa', inplace=True)

# Crea un índice temporal en df_unicas basado en 'localizacion_completa'
df_unicas.set_index('localizacion_completa', inplace=True)

# Actualiza 'coordenada_x' y 'coordenada_y' en df_resultado_final con los valores de df_unicas
df_resultado_final.update(df_unicas[['coordenada_x', 'coordenada_y']])

# Resetea el índice para volver al estado anterior
df_resultado_final.reset_index(inplace=True)
df_unicas.reset_index(inplace=True)



**Volvemos a comprobar cuantas coordenadas no ha encontrado**

In [22]:
nan_coordenada_x = df_resultado_final['coordenada_x'].isna().sum()
nan_coordenada_y = df_resultado_final['coordenada_y'].isna().sum()

print(f"Número de valores NaN en 'coordenada_x': {nan_coordenada_x}")
print(f"Número de valores NaN en 'coordenada_y': {nan_coordenada_y}")

Número de valores NaN en 'coordenada_x': 5441
Número de valores NaN en 'coordenada_y': 5441


**Aquí vamos a quitar las coordenadas que no ha encontrado y pasamos de UTM a latitud y longitud,
coordenadas que necesita folium**

In [23]:
# Filtrar filas donde 'coordenada_x' o 'coordenada_y' son NaN
df_coordenadas = df_resultado_final.dropna(subset=['coordenada_x', 'coordenada_y'])

# Definir el sistema de proyección UTM y WGS84, esto es para lo de UTM... que así venían los datos del ayto de Madrid ;)
utm_proj = Proj(proj='utm', zone=30, ellps='WGS84', preserve_units=False)
wgs84_proj = Proj(proj='latlong', datum='WGS84')

# Aquí convertimos de UTM a latitud y longitud
def utm_to_latlon(row):
    lon, lat = transform(utm_proj, wgs84_proj, row['coordenada_x'], row['coordenada_y'])
    return pd.Series({'latitud': lat, 'longitud': lon})

df_coordenadas[['latitud', 'longitud']] = df_coordenadas.apply(utm_to_latlon, axis=1)

df_coordenadas

C:\Users\34670\AppData\Local\Temp\ipykernel_18092\86459913.py:10: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(utm_proj, wgs84_proj, row['coordenada_x'], row['coordenada_y'])
C:\Users\34670\AppData\Local\Temp\ipykernel_18092\86459913.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coordenadas[['latitud', 'longitud']] = df_coordenadas.apply(utm_to_latlon, axis=1)
C:\Users\34670\AppData\Local\Temp\ipykernel_18092\86459913.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

,localizacion_completa,accidente_id,fecha,rango_horario,localizacion,numero,distrito,coordenada_x,coordenada_y,condicion,...,victimas,direccion_1,direccion_2,punto_negro,coordenada_x_nueva,coordenada_y_nueva,coordenada_x_nueva,coordenada_y_nueva,latitud,longitud
20,calle de segovia - paseo de la virgen del puer...,2010.435000,2010-01-16,1,calle de segovia - paseo de la virgen del puer...,,arganzuela,438928.763526,4.473716e+06,favorable,...,2.0,calle de segovia,paseo de la virgen del puerto ...,45,NaN,NaN,NaN,NaN,40.411827,-3.719797
24,avenida del planetario - calle de mendez alvar...,2010.562400,2010-06-21,14,avenida del planetario - calle de mendez alvar...,,arganzuela,441949.434244,4.471902e+06,favorable,...,1.0,avenida del planetario,calle de mendez alvaro ...,34,NaN,NaN,NaN,NaN,40.395695,-3.684032
26,avenida de la ciudad de barcelona ...,2010.100680,2010-11-11,20,avenida de la ciudad de barcelona ...,2,arganzuela,441419.606821,4.473250e+06,favorable,...,1.0,avenida de la ciudad de barcelona ...,None,32,NaN,NaN,NaN,NaN,40.407803,-3.690399
29,paseo de santa maria de la cabeza ...,2010.144500,2010-01-22,6,paseo de santa maria de la cabeza ...,1,arganzuela,441140.297521,4.473142e+06,favorable,...,3.0,paseo de santa maria de la cabeza ...,None,30,NaN,NaN,NaN,NaN,40.406811,-3.693680
41,glorieta de la ermita de la virgen de la soled...,2010.894400,2010-10-11,18,glorieta de la ermita de la virgen de la soled...,,barajas,450605.427171,4.479841e+06,favorable,...,1.0,glorieta de la ermita de la virgen de la soled...,None,41,NaN,NaN,NaN,NaN,40.467781,-3.582658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10457,"plaza. gabriel miro , 7",2024.001546,2024-01-15,17,plaza. gabriel miro,7,centro,439330.129000,4.473728e+06,NaN,...,NaN,plaza. gabriel miro,None,1,439330.129,4473727.919,439330.129,4473727.919,40.411961,-3.715068
10458,"calle nuñez de balboa , 86",2024.001547,2024-01-15,20,calle nuñez de balboa,86,salamanca,442146.054000,4.476054e+06,favorable,...,NaN,calle nuñez de balboa,None,1,442146.054,4476054.421,442146.054,4476054.421,40.433120,-3.682093
10459,"calle vigil - calle boyer, 2a",2024.001548,2024-01-15,19,calle vigil - calle boyer,2a,vicalvaro,450240.390000,4.472880e+06,desfavorable,...,NaN,calle vigil,calle boyer,1,450240.390,4472879.964,450240.390,4472879.964,40.405046,-3.586418
10460,"autov. m-30 09nc, 9nc7",2024.001550,2024-01-15,20,autov. m-30 09nc,9nc7,retiro,443468.671000,4.472932e+06,desfavorable,...,NaN,autov. m-30 09nc,None,1,443468.671,4472932.015,443468.671,4472932.015,40.405082,-3.666223


**Creamos una carpeta para los mapas, dentro de nuestro entorno de trabajo**

In [24]:
carpeta_mapas = 'mapas'
if not os.path.exists(carpeta_mapas):
    os.makedirs(carpeta_mapas)


**Vamos a buscar primero los distritos de Madrid para entender mejor los mapas**

In [25]:
valores_unicos_distrito = df_resultado_final['distrito'].unique()
print(valores_unicos_distrito)

['arganzuela' 'barajas' 'carabanchel' 'centro' 'chamartin' 'chamberi'
 'ciudad lineal' 'fuencarral-el pardo' 'hortaleza' 'latina'
 'moncloa-aravaca' 'moratalaz' 'puente de vallecas' 'retiro' 'salamanca'
 'san blas-canillejas' 'tetuan' 'usera' 'vicalvaro' 'villa de vallecas'
 'villaverde']


**Ahora con la librería geocoder buscamos cada distrito para obtener sus coordenadas**

In [26]:
# Lista de distritos
distritos = ['centro', 'carabanchel', 'latina', 'usera', 'moncloa-aravaca', 'moratalaz',
 'salamanca', 'villa de vallecas', 'villaverde', 'chamberi', 'chamartin',
 'hortaleza', 'ciudad lineal', 'retiro', 'fuencarral-el pardo', 'vicalvaro',
 'puente de vallecas', 'barajas', 'arganzuela', 'tetuan', 'san blas-canillejas']

# Diccionario para almacenar las coordenadas de los distritos
coordenadas_distritos = {}

# Bucle en geocoder para obtener las coordenadas
for distrito in distritos:
    g = geocoder.osm(f'{distrito}, Madrid')
    
    if g.ok:
        coordenadas_distritos[distrito] = g.latlng

#Imprimimos coordenadas
for distrito, coordenadas in coordenadas_distritos.items():
    print(f'Distrito: {distrito}, Coordenadas: {coordenadas}')

Distrito: centro, Coordenadas: [40.417652700000005, -3.70795469569446]
Distrito: carabanchel, Coordenadas: [40.3742112, -3.744676]
Distrito: latina, Coordenadas: [40.4035317, -3.736152]
Distrito: usera, Coordenadas: [40.383894, -3.7064459]
Distrito: moncloa-aravaca, Coordenadas: [40.43949485, -3.744206996100531]
Distrito: moratalaz, Coordenadas: [40.4059332, -3.6448737]
Distrito: salamanca, Coordenadas: [40.4270451, -3.6806024]
Distrito: villa de vallecas, Coordenadas: [40.3739576, -3.6121632]
Distrito: villaverde, Coordenadas: [40.3511231, -3.6997748]
Distrito: chamberi, Coordenadas: [40.4389621, -3.705302]
Distrito: chamartin, Coordenadas: [40.4589872, -3.6761288]
Distrito: hortaleza, Coordenadas: [40.4725491, -3.6425515]
Distrito: ciudad lineal, Coordenadas: [40.4484305, -3.650495]
Distrito: retiro, Coordenadas: [40.4111495, -3.6760566]
Distrito: fuencarral-el pardo, Coordenadas: [40.55634555, -3.7785905137518054]
Distrito: vicalvaro, Coordenadas: [40.4018377, -3.5950537]
Distrito: 

**Vamos a crear un mapa de los distritos de Madrid**

In [27]:
# Creamos un mapa de Madrid con un punto central y un nivel de zoom inicial
mapa_distritos_madrid = folium.Map(location=[40.416775, -3.703790], zoom_start=12)

# Añadir marcadores para cada distrito utilizando las coordenadas obtenidas previamente
# Es cierto que aquí nos marca un punto "central" en el distrito, no nos marca limitaciones de distritos
for distrito, coordenadas in coordenadas_distritos.items():
    # Añadimos un marcador para cada distrito
    folium.Marker(
        location=coordenadas,  # Aquí usa las coordenadas almacenadas en el diccionario anterior
        popup=distrito,  # Con esto el nombre del distrito aparecerá al hacer clic en el marcador
        icon=folium.Icon(color='red', icon='info-sign')  # Le ponemos color y datos al iconito
    ).add_to(mapa_distritos_madrid)

# Guardamos el mapa en un archivo HTML para visualizarlo en cualquier navegador (es interactivo, no una imagen)
nombre_archivo = "mapa_distritos_madrid.html"
mapa_distritos_madrid.save(nombre_archivo)

print(f"Mapa guardado como {nombre_archivo}")

# Con esto podemos ver el mapa aquí (ojo si contiene muchos datos no cargará... por ejemplo si hacemos todos los accidentes peta)
mapa_distritos_madrid


Mapa guardado como mapa_distritos_madrid.html


**Aquí vamos a utilizar CHOROPLETH y con un json descargado de la página del ayuntamiento de Madrid para graficar distritos**

In [28]:
# Cargamos el archivo shp que tenemos en la carpeta mapas
gdf = gpd.read_file('mapas/Distritos.shp')
gdf = gdf.to_crs(epsg=4326)  # Convertir a WGS84, necesario para cargar datos aquí

# Distritos de Madrid. Lo suyo sería encontrar los datos en un geojson... pero bueno, para otro día, los metemos a mano
distritos = ['centro', 'arganzuela', 'retiro', 'salamanca', 'chamartin', 'tetuan', 'chamberi', 'fuencarral-el pardo', 
 'moncloa-aravaca', 'latina', 'carabanchel', 'usera', 'puente-vallecas', 'moratalaz', 'ciudad lineal', 'hortaleza', 'villaverde',  
 'villa de vallecas', 'vicalvaro', 'san blas-canillejas','barajas']

# Añadimos los nombres de los distritos
gdf['nombre_distrito'] = distritos

# Creamos el mapa de Folium centrado en Madrid
m = folium.Map(location=[40.416775, -3.703790], zoom_start=11)

# Añadimos las líneas de los distritos al mapa y añadimos un marcador con el nombre del distrito
for _, row in gdf.iterrows():
    folium.GeoJson(row['geometry'], name=row['nombre_distrito']).add_to(m)
    folium.Marker(
        location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
        popup=row['nombre_distrito'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# Guardarmos
nombre_archivo = "mapas/mapa_distritos_madrid_folium.html"
m.save(nombre_archivo)

# Mostrar el mapa
m


** Vamos a graficar ahora únicamente los datos del 2024 que solo tenemos un mes para que se vea la cantidad de
accidentes que hay en Madrid y lo "feo" que ser vería con todos los puntos. Más que nada es para trasladar la cantidad
de puntos que podría tener si nos diera por graficar todos los accidentes desde el año 2010... ni se podría abrir**

In [30]:
df_2024 = df_coordenadas[pd.to_datetime(df_coordenadas['fecha']).dt.year == 2024]

df_2024 = df_2024.dropna(subset=['latitud', 'longitud'])

# Creamos un mapa centrado en una ubicación aproximada de Madrid
accidentes_madrid_2024 = folium.Map(location=[40.416775, -3.703790], zoom_start=12)

# Añadimos marcadores para cada accidente
for _, accidente in df_2024.iterrows():
    folium.Marker(
        location=[accidente['latitud'], accidente['longitud']],
        popup=f"Accidente ID: {accidente['accidente_id']}"
    ).add_to(accidentes_madrid_2024)

# Guardamos
nombre_archivo = "mapas/accidentes_madrid_2024.html"
accidentes_madrid_2024.save(nombre_archivo)

print(f"Mapa guardado como {nombre_archivo}")
accidentes_madrid_2024



Mapa guardado como mapas/accidentes_madrid_2024.html


**Ahora vamos a crear un mapa con el top 20 de puntos negros en Madrid por barrios desde el 2010 a la actudalidad**

También se podría hacer por años, meses... pero bueno, como es para bonito nos sobra...

Hay que hacer un merge para tener coordenadas y el ranking que están en dos df separados

In [49]:
# Realizar el merge entre df_agrupado y df_coordenadas usando 'localizacion_completa'
df_merged = df_agrupado.merge(df_coordenadas, on='localizacion_completa', how='left')

# Asegurarte de que df_merged está ordenado por 'ranking_punto_negro'
df_merged_sorted = df_merged.sort_values(by='ranking_punto_negro')

# Asegurarse de que no hay NaNs en las columnas 'latitud' y 'longitud'
top_puntos_por_distrito = top_puntos_por_distrito.dropna(subset=['latitud', 'longitud'])

# Crear un mapa de Folium centrado en Madrid
mapa_top_accidentes_madrid = folium.Map(location=[40.416775, -3.703790], zoom_start=11)

# Iterar
for _, punto in top_puntos_por_distrito.iterrows():
    folium.Marker(
        location=[punto['latitud'], punto['longitud']],
        popup=(
            f"Distrito: {punto['distrito_x']}<br>"
            f"Localización: {punto['localizacion_completa']}<br>"
            f"Ranking Punto Negro: {punto['ranking_punto_negro']}"
        ),
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(mapa_top_accidentes_madrid)

# Guardar
mapa_top_accidentes_madrid.save('mapas/mapa_top_accidentes_madrid.html')
mapa_top_accidentes_madrid
